# Metaprogramming
---
[< __GO BACK__](https://github.com/VCauthon/Summary-OpenEdg-Pyhon-PCPP1/blob/main/1.Advanced-OOP/5.Metaclasses/Introduction.ipynb)

### Introduction

Metaprogramming is a programming technique in which computer programs have __the ability to modify their own or other program's codes__. It may sound like an idea from a science fiction story, but the idea was born and implemented in the early 1960s.

For Python, code modifications can occure while the code is being executed, __and you might have already experienced__:
- `decorators`
- `overriding operators`
- `properties`

Tim Peters, the Python guru who authored the __Zen of Python__, expressed his feelings about metaclasses in the comp.lang.python newsgroup on 12/22/2002:

> __metaclasses__ - are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don't (the people who actually need them know with certainty that they need them, and don't need an explanation about why).

__Another example of metaprogramming is the metaclass concept__, which is one of the most advanced concepts presented in this course.

In Python, __a metaclass is a class whose instances are classes__. Just as an ordinary class defines the behavior of certain objects, a metaclass allows for the customization of class instantiation. __That is to say, we could understand it as the code that is executed, previously, when a class is instantiated__.

---

### What is the difference between a metaclass and decorating a class?

The functionality of the metaclass partly coincides with that of class decorators, but metaclasses act in a different way than decorators:

- __Decorators__: Bind the names of decorated functions or classes to new callable objects. Class decorators are applied when classes are instantiated.
- __Metaclasses__: Redirect class instantiations to dedicated logic, contained in metaclasses. Metaclasses are applied when class definitions are read to create classes, well before classes are instantiated.

The typical use cases for metaclasses:

- logging;
- registering classes at creation time;
- interface checking;
- automatically adding new methods;
- automatically adding new variables.

---

### The type() function

In other sections of the documentation the usefulness of the build-in type() function has been explained, as a reminder, this function allows to obtain, on an object, the class on which it has been instantiated.

However... see what happens if the same function is used on the class itself:

In [5]:
class Dog:
    def __init__(self, name, age):
        self.name = name
        self.age = age

dog_1 = Dog('Rex', 2)

# Let's see its type
print(type(dog_1))

# And with the class?
print(type(Dog))

<class '__main__.Dog'>
<class 'type'>


As we can see, by querying the type of the Dog class we have obtained type again (`<class 'type'>`)!

Taking into account the previous explanation, we can understand that the class Dog is just an instance of the special class type, which is the predefined metaclass in charge of creating classes.

> About classes being an instance of the type class, yes, in python everything is an object.

Other important details:
- The type of the metaclass `type` is `type` - no, __that is not a typo__.
- `type` is a class that generates classes defined by a programmer;
- __metaclasses__ are subclasses of the type class.

![Example code diagram](metaclasses.png)

---

### Special attributes of a class

Before going into detail on how to create a metaclass, let's do a little refresher on the special attributes that every class always acquires.

The attributes are:
- `__name__`: Returns the name of the object (either this is a class or its instance).
- `__class__`: Returns the class that generated the instance.
    - __NOTE__: Returns the same results from the `type()` function
- `__bases__`: Indicates from which classes the parent class is formed.
- `__dict__`: Returns, in a dictionary, the attributes of the object.

Example in code:

In [16]:
class Dog:
    pass

dog = Dog()
print('"dog" is an object of class named:', Dog.__name__)
print(''.center(50, '-'))
print(f'class "Dog" is an instance of: \n\t -__class__: {Dog.__class__} \n\t -type: {type(Dog)}')
print(f'class "dog" is an instance of: \n\t -__class__: {dog.__class__} \n\t -type: {type(dog)}')
print(''.center(50, '-'))
print('class "Dog" is  ', Dog.__bases__)
print(''.center(50, '-'))
print('class "Dog" attributes:', Dog.__dict__)
print('object "dog" attributes:', dog.__dict__)

"dog" is an object of class named: Dog
--------------------------------------------------
class "Dog" is an instance of: 
	 -__class__: <class 'type'> 
	 -type: <class 'type'>
class "dog" is an instance of: 
	 -__class__: <class '__main__.Dog'> 
	 -type: <class '__main__.Dog'>
--------------------------------------------------
class "Dog" is   (<class 'object'>,)
--------------------------------------------------
class "Dog" attributes: {'__module__': '__main__', '__dict__': <attribute '__dict__' of 'Dog' objects>, '__weakref__': <attribute '__weakref__' of 'Dog' objects>, '__doc__': None}
object "dog" attributes: {}


---

### Type and its arguments

So far, we have seen the function in action, however... we have only seen it executed with a single argument and this function... also allows three functions!

Methods of execution of the `type()` function:
- __1 Argument__: Returns the class that has been instantiated to create the argument sent.
- __3 Arguments__: Dynamically generates a class from the arguments sent, the usefulness of these is the following:
    1. Name of the class (this value becomes what the `__name__` attribute will return).
    2. Tuple with the base classes that will be used to generate the new class (what will inherit the class).
    3. A dictionary with the attributes that the class will be able to use.

__NOTE__: As we can see, the arguments received are the special attributes discussed in the previous section.

> We can state that Python, internally, uses the `type()` function when the keyword class is used to define a class.

As an example, we indicate how the class Dog that we have seen in the previous example could be created from the `type` function.

Demonstration:

In [15]:
Dog = type('Dog', (), {})

print('The class name is:', Dog.__name__)
print('The class is an instance of:', Dog.__class__)
print('The class is based on:', Dog.__bases__)
print('The class attributes are:', Dog.__dict__)


print(''.center(50, '-'))

dog = Dog()

print('The object is an instance of:', dog.__class__)
print('The object attributes are:', dog.__dict__)

The class name is: Dog
The class is an instance of: <class 'type'>
The class is based on: (<class 'object'>,)
The class attributes are: {'__module__': '__main__', '__dict__': <attribute '__dict__' of 'Dog' objects>, '__weakref__': <attribute '__weakref__' of 'Dog' objects>, '__doc__': None}
--------------------------------------------------
The object is an instance of: <class '__main__.Dog'>
The object attributes are: {}


---

### Creating a more realistic Dog class with type

In the previous example we have seen how you can create a class called Dog, however, this class is of little use since it doesn't really ... has no attributes that it can use.

Therefore, through type, we are going to include to the class Dog methods and variables that can be used by the instantiated objects.

The code is shown below:

In [31]:
class Animal:
    def eat(self):
        print("I'm eating")

    def sleep(self):
        print("I'm sleeping")

    def breathe(self):
        print("I'm breathing")


# Function that will be added as a method into the Dog class
@staticmethod
def bark():
    print("Woof!")


# Let's see what can do our dog
Dog = type('Dog', (Animal,), {'bark': bark})
dog = Dog()


# Methods inherited from Animal class (added as base class)
dog.eat()
dog.sleep()
dog.breathe()


# Function added as a method into the Dog class
dog.bark()


I'm eating
I'm sleeping
I'm breathing
Woof!


---

This way of creating classes, using the type function, is substantial for Python's way of creating classes using the class instruction:
- after the class instruction has been identified and the class body has been executed, the class = type(, , ) code is executed;
- the type is responsible for calling the `__call__` method upon class instance creation; this method calls two other methods:
    1. `__new__()`, responsible for creating the class instance in the computer memory.
    2. `__init__()`, responsible for object initialization.

![Alt text](instance_metaclass.png)

---

### Creating a metaclass

As we have seen so far, the type function is the one used by python to instantiate our custom class in order to generate instances of that class.

Now a few words from OpenEDG.

> It's important to remember that metaclasses are classes that are instantiated to get classes.

With this in mind we understand that we must create a class that is going to overlap between the type function (in charge of generating the class) and the custom class itself, that is to say, we have to create a metaclass.

> How can we achieve this?

If we take into account that type is the function in charge of generating classes ... we could say that, to create a metaclass, we must generate a class that inherits from type where the logic prior to the instantiation of the class is applied.

Here we would have an example:

In [32]:
# This is a metaclass!
class My_Meta(type):
    def __new__(mcs, name, bases, dictionary):
        obj = super().__new__(mcs, name, bases, dictionary)
        obj.custom_attribute = 'Added by My_Meta'
        return obj

# By passing My_Meta as the argument of "metaclass" type will call My_Meta instead of instantiating a new class
class My_Object(metaclass=My_Meta):
    pass

print(My_Object.__dict__)

{'__module__': '__main__', '__dict__': <attribute '__dict__' of 'My_Object' objects>, '__weakref__': <attribute '__weakref__' of 'My_Object' objects>, '__doc__': None, 'custom_attribute': 'Added by My_Meta'}


From the above code, we can understand the following:
1. My_Meta inherits from type.
2. My_Meta replaces method `__new__` (to be called by `__call__`)
3. The `__new__` method calls the `__new__` method of the parent class (which corresponds to the type function).
    - __mcs__: It uses `__new__` to reference itself as an object.
    - __Name, Inheritance and Attributes of the class__: name, base, dictionary: Arguments that type receives directly.
4. A custom attribute generated by the metaclass is included.
5. The instantiated object is returned for `__call__` to pass to `__init__`.

---

### Metaclass with logic

The following is an example of a metaclass that is able to include methods in case the defined class does not have them.

Code example:

In [34]:
def greetings(self):
    print('Just a greeting function, but it could be something more serious like a check sum')


# The metaclass append, if missing, the greetings method to the class
class My_Meta(type):
    def __new__(mcs, name, bases, dictionary):
        if 'greetings' not in dictionary:
            dictionary['greetings'] = greetings
        obj = super().__new__(mcs, name, bases, dictionary)
        return obj

# I'm missing the greetings method
class My_Class1(metaclass=My_Meta):
    pass

# I'm not missing the greetings method
class My_Class2(metaclass=My_Meta):
    def greetings(self):
        print('We are ready to greet you!')

myobj1 = My_Class1()
myobj1.greetings()
myobj2 = My_Class2()
myobj2.greetings()


Just a greeting function, but it could be something more serious like a check sum
We are ready to greet you!


---

[< __GO BACK__](https://github.com/VCauthon/Summary-OpenEdg-Pyhon-PCPP1/blob/main/1.Advanced-OOP/5.Metaclasses/Introduction.ipynb)